In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [2]:
df = pd.read_excel('../data/OMIEC_NEUROMORPHIC.xls')

In [ ]:
import requests
import time

df_with_responses = pd.DataFrame()

total = len(full_df['Abstract'])
total_time = 0

for index, row in full_df.iterrows():
    start_time = time.time()
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))
    payload = { 
        'prompt': """{
        'systemPrompt': 'RolePlay as a bot seeking for catalysts', 
        'user': "For the text I will input next, output only the catalyst materials used for the oxygen reduction reaction. Do not output precursors, electrolytes, and other kinds of materials, only the catalyst. The output must be only the catalyst material names separated by semicolons. Do not repeat the material name more than once. If no catalyst was found, output 'None'",
        'Assistant': 'Hello, I will tell the catalysts of any text you input next',
        'user_text':""" + row['Abstract'] + "}",
        "temperature":0.75,
        "topP":0.9,
        "maxTokens": 1000
    }
    #print(payload)
    response = requests.post('https://fumes-api.onrender.com/llama3', json=payload, stream=True)
    filtered_response = response.text.replace(' YOU CAN BUY ME COFFE! https://buymeacoffee.com/mygx', '')
    print(filtered_response)
    row['response'] = filtered_response
    df_with_responses = df_with_responses.append(row)
    time.sleep(2.5)
    total_time = total_time + time.time() - start_time
    print('Time for this loop:', time.time() - start_time, 'average loop time:', (total_time/(index+1)))
    #for chunk in response.iter_content(chunk_size=1024):  
    #    if chunk:
    #        print(chunk.decode('utf-8'))

In [ ]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
df_with_responses = pd.DataFrame()

total = len(df['Abstract'])
total_time = 0

for index, row in df.iterrows():
    start_time = time.time()
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers that are used in neuromorphic devices"
            },
            {
                "role": "user",
                "content": "For the text I will input next, output only polymers used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the polymers. The output must be only the polymer material names separated by semicolons. Do not repeat the polymer name more than once. If no polymer was found, output 'None'\""
            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": "\"\" + row['Abstract'] + "
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")